<a href="https://colab.research.google.com/github/sudht/NLP/blob/master/week5_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


<h2> 1.   개인 구글 드라이브와 colab 연동 </h2>

In [2]:
!pip install sklearn-crfsuite

<h2> 1.   라이브러리 설치 </h2>

In [3]:
import os
import sklearn_crfsuite
from sklearn_crfsuite import metrics


# 파일 경로
file_path = "/gdrive/My Drive/colab/week 5/spacing_data.txt"

# "spacing_data.txt" 파일을 읽고 lines에 읽은 데이터를 저장
with open(file_path, "r", encoding="utf8") as inFile:
    lines = inFile.readlines()

# 데이터를 음절로 이루어진 문장과 정답 값으로 나누어 저장
datas = []
for line in lines:
    pieces = line.strip().split("\t")
    eumjeol_sequence, label = pieces[0].split(), pieces[1].split()
    datas.append((eumjeol_sequence, label))
    
number_of_total_datas = len(datas)
number_of_train_datas = int(number_of_total_datas*0.9)
number_of_test_datas = number_of_total_datas-number_of_train_datas

print("전체 데이터 개수 : " + str(number_of_total_datas))
print("학습 데이터 개수 : " + str(number_of_train_datas))
print("평가 데이터 개수 : " + str(number_of_test_datas))
print()

train_datas, test_datas = [], []
for index, data in enumerate(datas):
    if(index < number_of_train_datas):
        train_datas.append(data)
    else:
        test_datas.append(data)
          
print("train_datas 개수 : " + str(len(train_datas)))          
print("test_datas 개수 : " + str(len(test_datas)))

전체 데이터 개수 : 1000
학습 데이터 개수 : 900
평가 데이터 개수 : 100

train_datas 개수 : 900
test_datas 개수 : 100


<pre>
<h2> 1. "spacing_data.txt"로 부터 데이터를 읽고 datas에 저장 </h2>
  1.1 입력 데이터 형태
    예시) 나 는 사 과 가 좋 아 tab B I B I I B I
    
    "나는 사과가 좋아" 라는 문장이 있을 때, 공백을 기준으로 나눈 "나는", "사과가", "좋아"를 각각 어절이라고 함
    각 어절의 시작음절을 "B"로 나머지를 "I"로 태깅하였고 문장과 라벨을 탭으로 구분해놓은 상태

  1.2 데이터 변환

    입력 데이터를 읽고 탭을 기준으로 문장과 라벨을 분리
    문장을 공백을 기준으로 나눈다
      예시)"나 는 사 과 가 좋 아" -> ["나", "는", "사", "과", "가", "좋", "아"]
    문장과 라벨을 튜플형태로 datas 리스트에 넣는다
    datas = [ ( ["나", "는", "사", "과", "가", "좋", "아"], ["B", "I", "B", "I", "I", "B", "I"] ), ( ... ), ... ]
    
<h2> 2. 전체 데이터를 9:1 비율에 맞추어 학습, 평가 데이터로 나누기 </h2>
  2.1 train_datas 리스트와 test_datas 리스트에 나누어 저장
</pre>

In [11]:
# sent2feature 함수 구현하기
def sent2feature(eumjeol_sequence):
  diclist = []
  lastIndex = len(eumjeol_sequence)
  for index, word in enumerate(eumjeol_sequence):
    dic = {"BOS": False, "EOS": False, "WORD": word, "IS_DIGIT": word.isdigit()}
    
    # BOS, EOS
    if index == 0:
      dic["BOS"] = True
    elif index == lastIndex-1:
      dic["EOS"] = True
    
    # -2_WORD, -1_WORD
    if index-1 >= 0:
      if index-2 >= 0:
        dic["-2_WORD"] = eumjeol_sequence[index-2]
      dic["-1_WORD"] = eumjeol_sequence[index-1]
      
    # +2_WORD, +1_WORD
    if index+1 <= lastIndex-1:
      if index+2 <= lastIndex-1:
        dic["+2_WORD"] = eumjeol_sequence[index+2]
      dic["+1_WORD"] = eumjeol_sequence[index+1]
       
    diclist.append(dic)
  #print(diclist)
  return diclist;
  
# train
train_x, train_y = [], []
for eumjeol_sequence, label in train_datas:
  train_x.append(sent2feature(eumjeol_sequence))
  train_y.append(label)
  
print(len(train_x[0]))
print(len(train_y[0]))

print(train_x[0])
print(train_y[0])

test_x, test_y = [], []
for eumjeol_sequence, label in test_datas:
  test_x.append(sent2feature(eumjeol_sequence))
  test_y.append(label)

27
27
[{'BOS': True, 'EOS': False, 'WORD': '약', 'IS_DIGIT': False, '+2_WORD': '장', '+1_WORD': '속'}, {'BOS': False, 'EOS': False, 'WORD': '속', 'IS_DIGIT': False, '-1_WORD': '약', '+2_WORD': '소', '+1_WORD': '장'}, {'BOS': False, 'EOS': False, 'WORD': '장', 'IS_DIGIT': False, '-2_WORD': '약', '-1_WORD': '속', '+2_WORD': '인', '+1_WORD': '소'}, {'BOS': False, 'EOS': False, 'WORD': '소', 'IS_DIGIT': False, '-2_WORD': '속', '-1_WORD': '장', '+2_WORD': '신', '+1_WORD': '인'}, {'BOS': False, 'EOS': False, 'WORD': '인', 'IS_DIGIT': False, '-2_WORD': '장', '-1_WORD': '소', '+2_WORD': '라', '+1_WORD': '신'}, {'BOS': False, 'EOS': False, 'WORD': '신', 'IS_DIGIT': False, '-2_WORD': '소', '-1_WORD': '인', '+2_WORD': '호', '+1_WORD': '라'}, {'BOS': False, 'EOS': False, 'WORD': '라', 'IS_DIGIT': False, '-2_WORD': '인', '-1_WORD': '신', '+2_WORD': '텔', '+1_WORD': '호'}, {'BOS': False, 'EOS': False, 'WORD': '호', 'IS_DIGIT': False, '-2_WORD': '신', '-1_WORD': '라', '+2_WORD': '커', '+1_WORD': '텔'}, {'BOS': False, 'EOS': False, 'WORD

<pre>
<h2> 1. 문장의 각 음절을 crf 모델의 입력으로 사용 할 수 있도록 자질화 </h2>
  "BOS" : 시작 음절인지 여부
  "EOS" : 마지막 음절인지 여부
  "WORD" : 기준 음절
  "IS_DIGIT" : 기준 음절이 숫자인지 여부
  "-1_WORD" : 기준 음절의 왼쪽 첫번째 음절
  "-2_WORD" : 기준 음절의 왼쪽 두번째 음절
  "+1_WORD" : 기준 음절의 오른쪽 첫번째 음절
  "+2_WORD" : 기준 음절의 오른쪽 두번째 음절

  1.1 예시) ["나", "는", "사", "과", "가", "좋", "아"]
            ->
            [ { "BOS":True, "EOS":False, "WORD":"나", "IS_DIGIT":False, "+1_WORD":"는", "+2_WORD":"사" },
    { "BOS":False, "EOS":False, "WORD":"는", "IS_DIGIT":False, "-1_WORD":"나", "+1_WORD":"사", "+2_WORD":"과" },
    { "BOS":False, "EOS":False, "WORD":"사", "IS_DIGIT":False, "-2_WORD":"나", "-1_WORD":"는", "+1_WORD":"과", "+2_WORD":"가" }, ... ]
  
    나 -> { "BOS":True, "EOS":False, "WORD":"나", "IS_DIGIT":False, "+1_WORD":"는", "+2_WORD":"사" }
    는 -> { "BOS":False, "EOS":False, "WORD":"는", "IS_DIGIT":False, "-1_WORD":"나", "+1_WORD":"사", "+2_WORD":"과" }
    사 -> { "BOS":False, "EOS":False, "WORD":"사", "IS_DIGIT":False, "-2_WORD":"나", "-1_WORD":"는", "+1_WORD":"과", "+2_WORD":"가" }
    과 -> { "BOS":False, "EOS":False, "WORD":"과", "IS_DIGIT":False, "-2_WORD":"는", "-1_WORD":"사", "+1_WORD":"가", "+2_WORD":"좋" }
    가 -> { "BOS":False, "EOS":False, "WORD":"가", "IS_DIGIT":False, "-2_WORD":"사", "-1_WORD":"과", "+1_WORD":"좋", "+2_WORD":"아" }
    좋 -> { "BOS":False, "EOS":False, "WORD":"좋", "IS_DIGIT":False, "-2_WORD":"과", "-1_WORD":"가", "+1_WORD":"아" }
    아 -> { "BOS":False, "EOS":True, "WORD":"아", "IS_DIGIT":False, "-2_WORD":"가", "-1_WORD":"좋" }
<h2> 2. 자질화한 데이터와 해당 데이터의 라벨을 분리하여 각 리스트에 저장 </h2>
  학습 데이터 -> train_x(자질화한 데이터), train_y(각 데이터의 정답 라벨)에 저장
  평가 데이터 -> test_x(자질화한 데이터), test_y(각 데이터의 정답 라벨)에 저장
  
  2.1 예시)
    train_x -> [
    
    [ { "BOS":True, "EOS":False, "WORD":"나", "IS_DIGIT":False, "+1_WORD":"는", "+2_WORD":"사" },
    { "BOS":False, "EOS":False, "WORD":"는", "IS_DIGIT":False, "-1_WORD":"나", "+1_WORD":"사", "+2_WORD":"과" },
    { "BOS":False, "EOS":False, "WORD":"사", "IS_DIGIT":False, "-2_WORD":"나", "-1_WORD":"는", "+1_WORD":"과", "+2_WORD":"가" }, ... ],
    
    [ ... ],
    
    ...
    
    ]
    
    train_y -> [
    
    [ "B", "I", "B", "I", "I", "B", "I" ],
    
    [ ... ],
    
    ...
    
    
    ]

</pre>

In [12]:
crf = sklearn_crfsuite.CRF(
        algorithm='lbfgs',
        c1=0.1,
        c2=0.1,
        max_iterations=100,
        all_possible_transitions=True
    )
crf.fit(train_x, train_y)

CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

<h2> 1. train_x, train_y를 이용하여 crf 모델 학습 </h2>

In [26]:
def makeString(x, y):
  test_datas_str = ""
  for index, eumjeol_sequence in enumerate(x):
    if y[index] == 'B':
      test_datas_str += " "
    test_datas_str += str(eumjeol_sequence)
  print(test_datas_str)
  

# show_predict_result 함수 구현하기
def show_predict_result(test_datas, pred_y):
  for index, x in enumerate(test_datas):
    y = pred_y[index]
    print("정답 문장 :",end='')
    makeString(x[0], x[1])
    print("출력 문장 :",end='')
    makeString(x[0], y)
    print()
    
labels = list(crf.classes_)
pred_y = crf.predict(test_x)

print("Accuracy score : " + str(metrics.flat_accuracy_score(test_y, pred_y)))
print()

print("10개의 데이터에 대한 모델 출력과 실제 정답 비교")
print()

show_predict_result(test_datas[:10], pred_y[:10])

Accuracy score : 0.9000381533765738

10개의 데이터에 대한 모델 출력과 실제 정답 비교

정답 문장 : 1914- 18년의 전쟁은 인류를 통합시킨 최초의 공통분모였다.
출력 문장 : 1914-18년의 전쟁은 인류를통 합시킨 최초의 공통분 모였다.

정답 문장 : 하지만 이 전쟁은 죽음을 통해 인류를 통합시켰다.
출력 문장 : 하지만이 전쟁은 죽음을 통해 인류를통 합시켰다.

정답 문장 : 사라예보에서 한 세르비아인이 쏜 총 한발이 합스부르크가의 계승자를 죽였다.
출력 문장 : 사라 예보에서 한 세르 비아인이 쏜총 한발이 합스부르크가의 계승 자를 죽였다.

정답 문장 : 이 암살행위는 국지적인 민족주의들과 세계적인 제국주의들이 충돌하는 분쟁지역에서 저질러졌다.
출력 문장 : 이암 살행 위는 국지적인 민족주의 들과 세계적인 제국주의 들이 충돌하는 분쟁 지역에서 저질러졌다.

정답 문장 : 오토만제국의 점진적인 해체는 민족주의의 독기를 발산하는 동시에 오스트리아, 헝가리와 독일, 영국, 프랑스의 탐욕을 자극했다.
출력 문장 : 오토만 제국의 점진 적인 해체는 민족주의의 독기를 발산하는 동시에 오스트리아, 헝가리와 독일, 영국, 프랑스의 탐욕을 자극했다.

정답 문장 : 이렇게 해서 발칸 반도의 한 외진 장소에서 벌어진 국지적인 테러 행위는 일련의 긴박한 반응을 불러 일으키면서 전 유럽에 영향을 미쳤을 뿐만 아니라 이번에는 아시아와 아프리카 식민지들, 일본, 그리고 이어서 미국과 멕시코까지 끌어들였다.
출력 문장 : 이렇게 해서 발칸 반도의 한외진 장소에서 벌어 진국지 적인테러행 위는 일련의 긴박한 반응을 불러일으키면서 전 유럽에 영향을 미쳤을 뿐만 아니라이 번에는 아시아와 아프리카 식민지들, 일본, 그리고 이어서 미국과 멕시코까지 끌어들였다.

정답 문장 : 전쟁의 물결이 지구상의 모든 대양으로 밀려드는 동안 캐나다인들과 미국인들, 오스트레일리아인들, 세네갈인들, 알제리인들, 모로코인들, 안남(安南)인들은 연합군 깃발을 휘날리며 유럽전

<pre>
<h2> 1. 학습한 모델을 test_x 데이터를 사용하여 평가 </h2>
<h2> 2. 모델의 출력 값과 정답 값을 이용하여 음절만으로 구성된 완전한 문장으로 변형 </h2>
  2.1 test_datas, pred_y 예시
    test_datas = [
    ( ["나", "는", "사", "과", "가", "좋", "아"], ["B", "I", "B", "I", "I", "B", "I"] ),
    
    ( ... ),
    
    ...
    
    ]
    
    pred_y = [
    
    ["B", "B", "B", "I", "I", "I", "I"],
    
    [ ... ],
    
    ...
    
    ]
    
    위의 문장을 기준으로한 변형 예시
    
    "나", "는", "사", "과", "가", "좋", "아" -> 나는 사과가 좋아 (정답 기준으로 변형)
    "나", "는", "사", "과", "가", "좋", "아" -> 나 는 사과가좋아 (모델 출력 기준으로 변형)
  
  2.2 최종 출력 예시
  
    정답 문장 : 1914- 18년의 전쟁은 인류를 통합시킨 최초의 공통분모였다.
    출력 문장 : 19 14- 18년의 전쟁은 인류를 통합시킨 최초의 공통 분모였다.

    정답 문장 : 하지만 이 전쟁은 죽음을 통해 인류를 통합시켰다.
    출력 문장 : 하지만이 전쟁은 죽음을 통해 인류를 통합시켰다.

    정답 문장 : 사라예보에서 한 세르비아인이 쏜 총 한발이 합스부르크가의 계승자를 죽였다.
    출력 문장 : 사라 예보에서 한세르비아인이 쏜총한 발이 합스부르크가의 계승자를 죽였다.
    
    ...
  
  
</pre>
